## Pega as antenas de Santa Catarina
<i>font:</i> http://www.telecocare.com.br/telebrasil/erbs/

In [1]:
from selenium import webdriver
import pymysql, warnings
import bs4

In [2]:
driver = webdriver.Chrome(executable_path='./webdriver/chromedriver')

In [3]:
driver.get('http://www.telecocare.com.br/telebrasil/erbs/')

In [4]:
htmlPage = bs4.BeautifulSoup(driver.page_source)

In [5]:
scripts = htmlPage.find_all('script',{'type':'text/javascript'})

In [6]:
content = []
for string in scripts[2].text.split('\t'):
    if '<p>' in string or 'LatLng' in string:
        content.append(string)

In [7]:
def cleanContent(content, type_c):
    if type_c == 'lat_lng':
        coordinates = content[content.find('(')+1:content.find(')')] 
        coordinates = coordinates.replace(' ','')
        lat, lng = coordinates.split(',')
        return lat, lng
    
    elif type_c == 'operator':
        operator = content[content.find('/b')+3:content.find('</p>')]
        return operator
    
    elif type_c == 'address':
        address = content[content.find('/b')+3:content.find('</p>')]
        return address
    
    elif type_c == 'citie':
        citie = content[content.find('/b')+3:content.find('</p>')]
        return citie

In [8]:
list_antennas = []
for antenna in range(1, len(content), 7):
    antenas = {'lat': '', 'lng': '', 'address': '', 'operator': '', 'citie': ''}
    
    # pega os dados brutos sobre cada antena
    lat_lng = content[antenna]
    operadora = content[antenna+1]
    endereco = content[antenna+2]
    municipio = content[antenna+4]
    
    #trata os dados de coordenadas
    lat, lng = cleanContent(lat_lng, 'lat_lng')
    
    #trata os dados da operadora
    operadora = cleanContent(operadora, 'operator')
    
    #trata os dados de endereco
    endereco = cleanContent(endereco, 'address')
    
    #trata os dados do municipio
    municipio = cleanContent(municipio, 'citie')
    
    antenas['lat'], antenas['lng'], antenas['operator'], antenas['address'], antenas['citie'] = lat, lng, operadora, endereco, municipio
    
    list_antennas.append(antenas)

In [163]:
class Database:
    def connect(self):
        trying = True
        while trying:
            try:
                self.db = pymysql.connect(host='instagram-bot.ccmmdvzb8vos.us-west-2.rds.amazonaws.com', user='admin', passwd='54203538', db='antenas', autocommit=True)
                trying = False
                return self.db
            except BaseException as error:
                if str(error).startswith('(2003,'):
                    trying = True
                else:
                    print('Error ocurred in: DB - connect --> {}'.format(error))
    
    def startCursor(self):
        self.cursor = self.db.cursor()
        warnings.simplefilter('ignore')
        return self.cursor
    
    def commitChanges(self):
        self.db.commit()

    def closeConnection(self):
        self.cursor.close()
        
    def saveAntenna(self, list_antennas):
        #recebe um dicionario com dados sobre o perfil
        for antenna in list_antennas:
            try:
                self.connect()
                self.startCursor()
                query =  'insert into antenas(address, citie, lat, lng, operator) values(%s,%s,%s,%s,%s)'
                self.cursor.execute(query, (antenna['address'], antenna['citie'], antenna['lat'], antenna['lng'], antenna['operator']))
                self.cursor.close()
            except BaseException as error:
                print('Error ocurred in: DB - saveAntenna --> type: {}, error: {}'.format(type(error), error))

In [164]:
database = Database()

In [40]:
database.saveAntenna(list_antennas)

In [23]:
import pickle

In [36]:
with open('antenas.data', 'wb') as _file:
    pickle.dump(list_antennas, _file)

In [37]:
with open('antenas.data', 'rb') as f:
    nova = pickle.load(f)